In [1]:
!pip install transformers torch datasets soundfile librosa

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 43.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 100.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install -U transformers accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.3.0
    Uninstalling accelerate-1.3.0:
      Successfully uninstalled accelerate-1.3.0


In [3]:
import torch
from transformers import WhisperProcessor, WhisperForConditionalGeneration, BitsAndBytesConfig
import time
import psutil
import os
from datasets import load_dataset

device = "cuda" if torch.cuda.is_available() else "cpu"
model_size = "medium"
processor = WhisperProcessor.from_pretrained(f"openai/whisper-{model_size}")
model = WhisperForConditionalGeneration.from_pretrained(f"openai/whisper-{model_size}").to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.99k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.06G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

Downloading the  entire dataset takes around 30 mins, so we use a single audio file for benchmarking

In [4]:
from datasets import load_dataset #datasets is a library in the huggingface ecosystem

dataset = load_dataset(
    "librispeech_asr",
    "clean",
    split="validation",
    streaming=True,#we don wanna access the whole dataset at once ,but rather access it piece by piece when needed
    trust_remote_code=True, #this is a flag, that we acknowledge the risk and willing to run the coee
    #Why Code? Datasets need special instructions to load correctly, like how to read their files or organize their data.
)
sample = next(iter(dataset))
sample_audio = sample["audio"]["array"]
sample_rate = sample["audio"]["sampling_rate"]

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

librispeech_asr.py:   0%|          | 0.00/11.4k [00:00<?, ?B/s]

the audio files chosen are random, that is each time a different file will be present at position 0


In [27]:
def transcribe(audio, processor, model, device):
    input_features = processor(audio, sampling_rate=sample_rate, return_tensors="pt").input_features.to(device)
    input_features = input_features.to(torch.float16)
    start_time = time.time()
    predicted_ids = model.generate(input_features)
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
    end_time = time.time()
    latency = end_time - start_time
    return transcription, latency

def get_cpu_memory_usage():
    process = psutil.Process()
    memory_info = process.memory_info()
    return memory_info.rss / (1024 * 1024)


QUANTIZATION

In [10]:
# INT8 Quantization (CPU)
if device == "cpu":
    quantized_model_int8 = torch.quantization.quantize_dynamic(
        model, {torch.nn.Linear}, dtype=torch.qint8
    )
    transcription_int8, latency_int8 = transcribe(sample_audio, processor, quantized_model_int8, device)
    cpu_memory_int8 = get_cpu_memory_usage()
    print(f"INT8 Quantized CPU Transcription: {transcription_int8[0]}")
    print(f"INT8 Quantized CPU Latency: {latency_int8:.4f} seconds")
    print(f"INT8 Quantized CPU Memory: {cpu_memory_int8:.4f} MB")

In [28]:
# 4-bit Quantization (GPU)
if device == "cuda":
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4",
    )
    quantized_model_4bit = WhisperForConditionalGeneration.from_pretrained(
        f"openai/whisper-{model_size}", quantization_config=quantization_config
    ).to(device)
    transcription_4bit, latency_4bit = transcribe(sample_audio, processor, quantized_model_4bit, device)
    gpu_peak_memory = torch.cuda.max_memory_allocated(device=device)
    print(f"4-bit Quantized GPU Transcription: {transcription_4bit[0]}")
    print(f"4-bit Quantized GPU Latency: {latency_4bit:.4f} seconds")
    print(f"4-bit Quantized GPU Peak Memory: {gpu_peak_memory / (1024 * 1024):.2f} MB")

`low_cpu_mem_usage` was None, now default to True since model is quantized.


4-bit Quantized GPU Transcription:  He was in a fevered state of mind, owing to the blight his wife's action threatened to cast upon his entire future.
4-bit Quantized GPU Latency: 2.6260 seconds
4-bit Quantized GPU Peak Memory: 4301.86 MB


In [16]:
num_runs = 10
latencies = []
cpu_memory_usages = []

if device == "cuda":
    torch.cuda.reset_peak_memory_stats(device=device)
    start_memory = torch.cuda.memory_allocated(device=device)

    for _ in range(num_runs):
        transcription, latency = transcribe(sample_audio, processor, model, device)
        latencies.append(latency)

    end_memory = torch.cuda.memory_allocated(device=device)
    peak_memory = torch.cuda.max_memory_allocated(device=device)
    memory_usage = end_memory - start_memory

    avg_latency = sum(latencies) / num_runs
    print(f"GPU Transcription: {transcription[0]}")
    print(f"GPU Average Latency: {avg_latency:.4f} seconds")
    print(f"GPU Memory Usage: {memory_usage / (1024 * 1024):.2f} MB")
    print(f"GPU Peak Memory Usage: {peak_memory / (1024 * 1024):.2f} MB")

else:
    for _ in range(num_runs):
        start_cpu_memory = get_cpu_memory_usage()
        transcription, latency = transcribe(sample_audio, processor, model, device)
        end_cpu_memory = get_cpu_memory_usage()
        cpu_memory_usages.append(end_cpu_memory - start_cpu_memory)
        latencies.append(latency)

    avg_latency = sum(latencies) / num_runs
    avg_cpu_memory = sum(cpu_memory_usages) / num_runs

    print(f"CPU Transcription: {transcription[0]}")
    print(f"CPU Average Latency: {avg_latency:.4f} seconds")
    print(f"CPU Average Memory Usage: {avg_cpu_memory:.2f} MB")

GPU Transcription:  He was in a fevered state of mind, owing to the blight his wife's action threatened to cast upon his entire future.
GPU Average Latency: 1.3950 seconds
GPU Memory Usage: 0.00 MB
GPU Peak Memory Usage: 4103.10 MB


FOR BASELINE TRANSRIPTION INPUT SHOULD BE float16
FOR 4 BIT GPU QUANTIZATION INPUT SHOULD BE float32

In [17]:
!pip install auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 103.7 MB/s eta 0:00:00


In [18]:
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig

quantize_config = BaseQuantizeConfig(bits=4, group_size=128)  # Example settings

# Load the model and quantize it
model_gptq = AutoGPTQForCausalLM.from_pretrained(
    f"openai/whisper-{model_size}", quantize_config
)

#

/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:410: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:418: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
/usr/local/lib/python3.11/dist-packages/auto_gptq/nn_modules/triton_utils/kernels.py:461: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd(cast_inputs=torch.float16)


TypeError: whisper isn't supported yet.

Not all quantization libraries support all model architectures.
GPTQ:auto-gptq library Does not support Whisper yet

# AWSQ

In [22]:
!pip install awq

from awq import AutoAWQForCausalLM

ERROR: Could not find a version that satisfies the requirement awq (from versions: none)
ERROR: No matching distribution found for awq


ModuleNotFoundError: No module named 'awq'

In [23]:
!git clone https://github.com/mit-han-lab/awq.git
%cd awq
!pip install -r requirements.txt
!pip install .

Cloning into 'awq'...
fatal: could not read Username for 'https://github.com': No such device or address
[Errno 2] No such file or directory: 'awq'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


In [24]:
!git clone https://github.com/mit-han-lab/awq.git

Cloning into 'awq'...
fatal: could not read Username for 'https://github.com': No such device or address


In [25]:
%cd awq
!pip install -r requirements.txt
!pip install .

[Errno 2] No such file or directory: 'awq'
/content
ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt'
ERROR: Directory '.' is not installable. Neither 'setup.py' nor 'pyproject.toml' found.


Could not find the awq library

## VLLM

In [26]:
!pip install vllm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 343.3/343.3 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 114.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2